In [56]:
import cv2
import numpy as np

In [57]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [58]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [59]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [60]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    def forward(self,inp):
        out = self.fc1(inp)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [61]:
def train(net):
    global num_epochs
    for ep in range(num_epochs):
        for i,(images,labels) in enumerate(train_loader):
            images = Variable(images.view(-1,28*28))
            labels = Variable(labels)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                       %(ep+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

In [62]:
def get_accuracy(net,name):
    global test_loader
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images.view(-1, 28*28))
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    # Save the Model
    torch.save(net.state_dict(), str(name+'model')+'.pkl')



In [63]:
net = Net(input_size,hidden_size,num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
train(net)

Epoch [1/5], Step [100/600], Loss: 0.4667
Epoch [1/5], Step [200/600], Loss: 0.2215
Epoch [1/5], Step [300/600], Loss: 0.3113
Epoch [1/5], Step [400/600], Loss: 0.2085
Epoch [1/5], Step [500/600], Loss: 0.1936
Epoch [1/5], Step [600/600], Loss: 0.1374
Epoch [2/5], Step [100/600], Loss: 0.0466
Epoch [2/5], Step [200/600], Loss: 0.0666
Epoch [2/5], Step [300/600], Loss: 0.0800
Epoch [2/5], Step [400/600], Loss: 0.1298
Epoch [2/5], Step [500/600], Loss: 0.1581
Epoch [2/5], Step [600/600], Loss: 0.0638
Epoch [3/5], Step [100/600], Loss: 0.0553
Epoch [3/5], Step [200/600], Loss: 0.0352
Epoch [3/5], Step [300/600], Loss: 0.0955
Epoch [3/5], Step [400/600], Loss: 0.0490
Epoch [3/5], Step [500/600], Loss: 0.0873
Epoch [3/5], Step [600/600], Loss: 0.0490
Epoch [4/5], Step [100/600], Loss: 0.0611
Epoch [4/5], Step [200/600], Loss: 0.0542
Epoch [4/5], Step [300/600], Loss: 0.1475
Epoch [4/5], Step [400/600], Loss: 0.0240
Epoch [4/5], Step [500/600], Loss: 0.0519
Epoch [4/5], Step [600/600], Loss:

In [64]:
get_accuracy(net,'feedForward')

Accuracy of the network on the 10000 test images: 97 %


In [70]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
                nn.Conv2d(1,16,kernel_size=5, padding=2),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                nn.Conv2d(16,32,kernel_size=5, padding=2),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32,10)
    def forward(self,inp):
        out = self.layer1(inp)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
    

In [71]:
cnn = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)


In [76]:
def cnn_train():
    for ep in range(num_epochs):
        for i,(images,labels) in enumerate(train_loader):
            images = Variable(images)
            labels = Variable(labels)
            optimizer.zero_grad()
            outputs = cnn(images)
            loss = criterion(outputs,labels)
        
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                       %(ep+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
            

In [77]:
def cnn_accuracy():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images)
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    # Save the Model
    torch.save(cnn.state_dict(), 'cnnModel.pkl')

In [78]:
cnn_train()

Epoch [1/5], Step [100/600], Loss: 0.1274
Epoch [1/5], Step [200/600], Loss: 0.1015
Epoch [1/5], Step [300/600], Loss: 0.1087
Epoch [1/5], Step [400/600], Loss: 0.0526
Epoch [1/5], Step [500/600], Loss: 0.1374
Epoch [1/5], Step [600/600], Loss: 0.0247
Epoch [2/5], Step [100/600], Loss: 0.0293
Epoch [2/5], Step [200/600], Loss: 0.0357
Epoch [2/5], Step [300/600], Loss: 0.0287
Epoch [2/5], Step [400/600], Loss: 0.0392
Epoch [2/5], Step [500/600], Loss: 0.0179
Epoch [2/5], Step [600/600], Loss: 0.0094
Epoch [3/5], Step [100/600], Loss: 0.0522
Epoch [3/5], Step [200/600], Loss: 0.0492
Epoch [3/5], Step [300/600], Loss: 0.0432
Epoch [3/5], Step [400/600], Loss: 0.0188
Epoch [3/5], Step [500/600], Loss: 0.0575
Epoch [3/5], Step [600/600], Loss: 0.0680
Epoch [4/5], Step [100/600], Loss: 0.0392
Epoch [4/5], Step [200/600], Loss: 0.0081
Epoch [4/5], Step [300/600], Loss: 0.0047
Epoch [4/5], Step [400/600], Loss: 0.0504
Epoch [4/5], Step [500/600], Loss: 0.0111
Epoch [4/5], Step [600/600], Loss:

In [79]:
cnn_accuracy()

Accuracy of the network on the 10000 test images: 99 %


In [121]:
transform = transforms.Compose([
    transforms.Scale(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [122]:
trainres_dataset = dsets.CIFAR10(root='./data/',
                               train=True, 
                               transform=transform,
                               download=True)

testres_dataset = dsets.CIFAR10(root='./data/',
                              train=False, 
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
trainres_loader = torch.utils.data.DataLoader(dataset=trainres_dataset,
                                           batch_size=100, 
                                           shuffle=True)

testres_loader = torch.utils.data.DataLoader(dataset=testres_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


In [123]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [124]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [125]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [126]:
resnet = ResNet(ResidualBlock, [2,2,2,2])
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)

In [127]:
print "hello"

hello


In [130]:
for epoch in range(10):
    for i, (images, labels) in enumerate(trainres_loader):
        images = Variable(images)
        labels = Variable(labels)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [%d/%d], Iter [%d/%d] Loss: %.4f" %(epoch+1, 80, i+1, 500, loss.data[0]))

    # Decaying Learning Rate
    if (epoch+1) % 20 == 0:
        lr /= 3
        optimizer = torch.optim.Adam(resnet.parameters(), lr=lr) 

Epoch [1/80], Iter [100/500] Loss: 1.0982
Epoch [1/80], Iter [200/500] Loss: 1.2597
Epoch [1/80], Iter [300/500] Loss: 0.9493
Epoch [1/80], Iter [400/500] Loss: 1.0262
Epoch [1/80], Iter [500/500] Loss: 1.0159
Epoch [2/80], Iter [100/500] Loss: 1.0057
Epoch [2/80], Iter [200/500] Loss: 1.1290
Epoch [2/80], Iter [300/500] Loss: 0.8771
Epoch [2/80], Iter [400/500] Loss: 0.9684
Epoch [2/80], Iter [500/500] Loss: 0.8919
Epoch [3/80], Iter [100/500] Loss: 0.9478
Epoch [3/80], Iter [200/500] Loss: 0.8078
Epoch [3/80], Iter [300/500] Loss: 0.9069
Epoch [3/80], Iter [400/500] Loss: 0.7679
Epoch [3/80], Iter [500/500] Loss: 0.7568
Epoch [4/80], Iter [100/500] Loss: 0.9381
Epoch [4/80], Iter [200/500] Loss: 0.7242
Epoch [4/80], Iter [300/500] Loss: 0.7873
Epoch [4/80], Iter [400/500] Loss: 0.9324
Epoch [4/80], Iter [500/500] Loss: 0.5722
Epoch [5/80], Iter [100/500] Loss: 0.6943
Epoch [5/80], Iter [200/500] Loss: 0.6045
Epoch [5/80], Iter [300/500] Loss: 0.6436
Epoch [5/80], Iter [400/500] Loss:

In [132]:
correct = 0
total = 0
for images, labels in testres_loader:
    images = Variable(images)
    outputs = resnet(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(resnet.state_dict(), 'resnet.pkl')

Accuracy of the model on the test images: 78 %
